# Bagging

Razvijemo razred za tehniko bagging, oziroma [bootstrap aggregating](https://en.wikipedia.org/wiki/Bootstrap_aggregating), ki lahko izboljša stabilnost in točno napovednih metod, katerih struktura modela je lahko precej odvisna od vzorca podatkov. To je, katerih struktura zgrajenega modela se pri manjših spremembah v učnih podatkih lahko precej spremeni. Primer take metode so klasifikacijska in regresijska drevesa, lahko pa bagging pomaga tudi kakšnim drugim tehnikam. Bagging si ogledamo tudi kot primer ansamblov, torej metod, ki temeljijo na gradnji množice modelov in potem s povprečenjem ali glasovanjem predlagajo vrednost razreda.

Najprej preberemo podatke.

In [1]:
import Orange
from Orange.preprocess.preprocess import Preprocess
import numpy as np

In [2]:
class RemoveConstant(Preprocess):
    def __call__(self, data):
        oks = np.min(data.X, axis=0) != np.max(data.X, axis=0)
        atts = [a for a, ok in zip(data.domain.attributes, oks) if ok]
        domain = Orange.data.Domain(atts, data.domain.class_vars)
        return Orange.data.Table(domain, data)

rc = RemoveConstant()

In [3]:
orig = Orange.data.Table("digits-sample.csv")
orig = Orange.data.Table("seven-nine.csv")
# orig = Orange.data.Table("promoters")
# orig = Orange.data.Table("all-1000.csv")
data = rc(orig)
len(data), len(data.domain.attributes)

(500, 523)

Na podatkih ovrednotimo nekaj osnovnih klasifikatorjev. Uporabimo klasifikacijska drevesa in logistično regresijo.

In [4]:
tree = Orange.classification.SimpleTreeLearner()
lr = Orange.classification.LogisticRegressionLearner()
maj = Orange.classification.MajorityLearner()

In [5]:
learners = [tree, lr, maj]
res = Orange.evaluation.CrossValidation(data, learners, k=5)

In [6]:
print("\n".join("%20s %5.2f" % (learner.name, score) for learner, score in zip(learners, Orange.evaluation.AUC(res))))

         simple tree  0.89
              logreg  0.93
            majority  0.50


Drevesa se ne obneseje najbolje, logistična regresija je boljša.

### Orange-ov razred za Bagging

Orange-ova shema za napovedne modele vedno vsebuje Learner in Model, torej, razred, ki je namenjen učenju in razred, ki je namenjen napovedovanju. Uporabimo razred in ne funkcije: iz njih izvedeni objekti si bodo morali zapomniti nekatere nastavitve in parametre, potem pa jih bomo uporabili pri učenju in napovedovanju. Razred za učenje (BaggedLearner) si bo moral zapomniti algoritem učenja, ki ga bomo uporabili na k vzorcih podatkov, razred za napovedovanje (BaggedModel) pa bomo inicializirali s seznamom k naučenih modelov.

BaggedLearner implementira vzorčenje po metodi stremena (angl. boosting). Po inicializaciji objekt kličemo s podatki. Klic nam vrne regresor, BaggedModel, ki hrani naučene modele. Ob klicu regresorja ta uporabi napovedne modele in izračuna povprečje napovedi, ki jih ti vrnejo. BaggedModel lahko kličemo z enim samim primerom, lahko pa mu tudi podtaknemo tabelo primerov. Pri slednjem nam bo klic tega objekta vrnil vektor napovedi.

Bagging bomo implementirali za regresijo, torej privzeli, da nam napovedni modeli vračajo realne vrednost. Pri klasifikaciji bi implementacija bila rahlo kompleksnejša, saj nam v Orange-u klasifikatorji lahko vračajo razrede ali pa njihove verjetnosti, odvisno od parametra, ki ga podamo pri klicu klasifikatorja.

In [7]:
class BaggedLearner(Orange.classification.Learner):
    """Bootstrap aggregating learner."""
    def __init__(self, learner, k=3):
        super().__init__()
        self.k = k # number of bootstrap samples and corresponding models
        self.learner = learner # base learner
        self.name = "bagged " + self.learner.name
    
    def fit_storage(self, data):
        """Return a bagged model inferred from the training set."""
        models = []
        for epoch in range(self.k):
            indices = np.random.randint(len(data), size=len(data))  # sample indices
            models.append(self.learner(data[indices]))  # model inference from a sample
        model = BaggedModel(data.domain, models)  # return a predictor that remembers a list of models
        model.name = self.name
        return model

class BaggedModel(Orange.classification.Model):
    """Bootstrap aggregating classifier."""
    def __init__(self, domain, models):
        super().__init__(domain)
        self.models = models  # a list of predictors
    
    def predict_storage(self, data, ret=Orange.classification.Model.Value):
        """Given a data instance or table of data instances returns predicted class."""
        p = np.mean([m(data, 1) for m in self.models], axis=0)
        return p / p.sum(axis=1, keepdims=True)

Uporabimo bagging nad drevesi in linearno regresijo. Točnost napovedi ocenimo s prečnim preverjanjem.

In [8]:
bag_tree = BaggedLearner(tree, k=50)
bag_lr = BaggedLearner(lr, k=50)

In [9]:
bt = bag_tree(data)
bt(data[0], 1)

array([[ 1.,  0.]])

In [10]:
bt(data[0:3], 1)

array([[ 1.  ,  0.  ],
       [ 0.03,  0.97],
       [ 1.  ,  0.  ]])

In [11]:
learners = [tree, bag_tree, lr, bag_lr]
learners = [tree, bag_tree]
res = Orange.evaluation.CrossValidation(data, learners, k=5)

In [12]:
print("\n".join("%20s %5.2f" % (learner.name, score) for learner, score in zip(learners, Orange.evaluation.AUC(res))))

         simple tree  0.89
  bagged simple tree  0.92


Bagging z drevesi se med zgornjimi dobro obnese. Zanimivo. Izboljšanje logistične regresije pa nam po drugi strani ni uspelo. Zakaj? Drevesa zgrajena na vzorčenih podatki se dejansko bistveno razlikujejo med sabo (kako bi to preverili?), razlike med dobljenimi modeli logistične regresije pa so majhne.

# Ponaključenje

Sledi finta. V angleščini bi se ji reklo *randomization*. Namreč, radi bi izboljšali tudi napovedi linearne regresije. In pri tem uporabili ansamble, oziroma niz napovednih modelov. Ampak, kot že rečeno, ti napovedni modeli se morajo med sabo razlikovati, sicer jih z baggingom ne bomo prav dosti izboljšali. Namesto vzorčenja podatkov bomo raje vzorčili atribute, in regresorje, na primer linearno regresiju, gradili na celotnih podatkih, ki pa bodo vsebovali samo manjši delež (p) originalnih atributov. Pričakujemo, da se bodo modeli linearne regresije, ki bodo zgrajeni na različnih domenah (različnih naborih atributov) med seboj dovolj razlikovali, da bo njihov ansambel uspešen.

Spodaj implementiramo samo razred za učenje (RandomizedLearner), za napovedovanje s povprečenjem napovedi večih modelov pa recikliramo kar BaggedModel. RandomizedLearner je podobno preprost kot BaggedLearner. Za izbor atributov oziroma konstrukcijo nove tabele z podmnožico originalnih atributov smo uporabili kar Orange-ov SelectRandomFeatures.

In [13]:
class RandomizedLearner(Orange.classification.Learner):
    """Ensamble learning through randomization of data domain."""
    def __init__(self, learner, k=3, p=0.5):
        super().__init__()
        self.k = k
        self.learner = learner
        self.name = "rand " + self.learner.name
        # a function to be used for random attribute subset selection
        self.selector = Orange.preprocess.fss.SelectRandomFeatures(k=p)
    
    def fit_storage(self, data):
        """Returns a bagged model with randomized regressors."""
        models = []
        for epoch in range(self.k):
            sample = self.selector(data) # data with a subset of attributes
            models.append(self.learner(sample))
        model = BaggedModel(data.domain, models)
        model.name = self.name
        return model

In [15]:
rnd_lr = RandomizedLearner(lr, k=50, p=0.2)
learners = [maj, lr, rnd_lr]
res = Orange.evaluation.CrossValidation(data, learners, k=5)

In [16]:
print("\n".join("%20s %5.2f" % (learner.name, score) for learner, score in zip(learners, Orange.evaluation.AUC(res))))

            majority  0.50
              logreg  0.93
         rand logreg  0.92


Tole deluje še kar ok, oziroma logistične regresije vsaj ne pokvari. Na kakšni drugi domeni bi lahko tudi pomagalo, oziroma se zna zgoditi, da ima ansambel ponaključenih logističnih regresij večjo točnost od logistične regresije same.

# Naključni gozd

Klasičen, ali morda celo najbolj tipičen primer ponaklučenja (randomizacije) so naključni gozdovi (angl. random forest). Tu je bil narejen poseg v algoritem učenja dreves, ki ob gradnji drevesa ne uporabi vač za razcep množice primerov najprimernejšega atributa, ampak atribut za razcep naključno izbere iz množice najprimernejši. Poleg uporabljenega vzorčanje z metodo stremena to še dodatno poskrbi za raznolikost dreves v gozdu.

Spodaj smo kot primer uporabili kar SimpleRandomForestLearner, ki je hitra C-jevska implementacija naključnih gozdov. Dokler ni scikit-learn odpravil nekaj bugov in pospešil svojo implementacijo, je bil SimpleRandomForestLearner morda najhitrejša implementacija naključnega gozda z vmesnikom v Pythonu. Orange zawrapa sicer tudi implementacijo iz sklearn-a (RandomForestLearner), ki pa meni, vsaj na tej domeni, včasih daje malce slabše rezultate od Orange-ovega.

In [17]:
skl_forest = Orange.classification.RandomForestLearner(n_estimators=50)
skl_forest.name = "skl forest"
forest = Orange.classification.SimpleRandomForestLearner(n_estimators=50)
learners = [tree, forest, skl_forest]
res = Orange.evaluation.CrossValidation(data, learners, k=5)

In [18]:
print("\n".join("%20s %5.2f" % (learner.name, score) for learner, score in zip(learners, Orange.evaluation.AUC(res))))

         simple tree  0.89
     simple rf class  0.93
          skl forest  0.95


In [19]:
a = np.array([[1,2], [3,4], [5,6]])

In [20]:
a/a.sum(axis=1, keepdims=True)

array([[ 0.33333333,  0.66666667],
       [ 0.42857143,  0.57142857],
       [ 0.45454545,  0.54545455]])